# package

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import datar.all as da

In [4]:
import json

In [5]:
import requests

In [7]:
from flatsplode import flatsplode

In [8]:
import mygene

# geneinfo

## human

In [287]:
# input_symbol = "PTH1R"
input_symbol = "IRF7"
# input_symbol = "T"
# input_symbol = ""

In [288]:
mg = mygene.MyGeneInfo()

In [289]:
genetable0 = mg.query("symbol:" + input_symbol, 
                      fields = "symbol, name, ensembl.gene, entrezgene, uniprot.Swiss-Prot, homologene.id", 
                      species = "human")

In [290]:
if genetable0["total"] == 0: 
    print("Please input the correct gene symbol.")

In [291]:
genetable00 = pd.DataFrame({
    "symbol": pd.Series([], dtype = object), 
    "name": pd.Series([], dtype = object), 
    "ensembl.gene": pd.Series([], dtype = object), 
    "entrezgene": pd.Series([], dtype = object), 
    "uniprot.Swiss-Prot": pd.Series([], dtype = object), 
    "homologene.id": pd.Series([], dtype = np.int64)
    }
)

In [292]:
genetable00

,symbol,name,ensembl.gene,entrezgene,uniprot.Swiss-Prot,homologene.id
,<object>,<object>,<object>,<object>,<object>,<int64>


In [354]:
genetable00.shape

(0, 6)

In [293]:
type(genetable00)

pandas.core.frame.DataFrame

In [294]:
genetable = pd.concat([genetable00, pd.DataFrame(list(flatsplode(genetable0["hits"][0])))]) >> \
    da.select("symbol", "name", "ensembl.gene", "entrezgene", "uniprot.Swiss-Prot", "homologene.id")

In [295]:
genetable

,symbol,name,ensembl.gene,entrezgene,uniprot.Swiss-Prot,homologene.id
,<object>,<object>,<object>,<object>,<object>,<int64>
0,IRF7,interferon regulatory factor 7,ENSG00000276561,3665,Q92985,128624
1,IRF7,interferon regulatory factor 7,ENSG00000185507,3665,Q92985,128624


In [355]:
genetable.shape

(2, 6)

In [296]:
type(genetable)

pandas.core.frame.DataFrame

In [297]:
gene_ensembls = genetable["ensembl.gene"]

In [298]:
gene_ensembls

0    ENSG00000276561
1    ENSG00000185507
Name: ensembl.gene, dtype: object

In [299]:
type(gene_ensembls)

pandas.core.series.Series

In [300]:
base_url = "https://api.platform.opentargets.org/api/v4/graphql"

In [301]:
query_geneinfo = """
    query target($ensemblId: String!) {
        target(ensemblId: $ensemblId) {
            id
            approvedSymbol
            approvedName
        }
    }
"""

In [302]:
for gene_ensembl in gene_ensembls:
    variables0 = {"ensemblId": gene_ensembl}
    r0 = requests.post(base_url, json = {"query": query_geneinfo, "variables": variables0})
    res0 = json.loads(r0.text)
    if not res0["data"]["target"]:
        continue 
    else:
        break

In [303]:
ensembl_gene_ok = pd.DataFrame(list(flatsplode(res0["data"]["target"])))["id"][0]

In [304]:
ensembl_gene_ok

'ENSG00000185507'

In [356]:
type(ensembl_gene_ok)

str

In [305]:
genetable_ok = genetable >> \
    da.filter(da.f["ensembl.gene"] == ensembl_gene_ok) >> \
    da.rename(ensembl_gene_ID = "ensembl.gene", 
              entrezgene_ID = "entrezgene", 
              uniprot_ID = "uniprot.Swiss-Prot", 
              homologene_ID = "homologene.id")

In [306]:
genetable_ok

,symbol,name,ensembl_gene_ID,entrezgene_ID,uniprot_ID,homologene_ID
,<object>,<object>,<object>,<object>,<object>,<int64>
1,IRF7,interferon regulatory factor 7,ENSG00000185507,3665,Q92985,128624


In [358]:
genetable_ok.shape

(1, 6)

In [359]:
type(genetable_ok)

datar.core.tibble.Tibble

In [307]:
def genetable_ok_def():
    genetable = pd.concat([genetable00, pd.DataFrame(list(flatsplode(genetable0["hits"][0])))]) >> \
        da.select("symbol", "name", "ensembl.gene", "entrezgene", "uniprot.Swiss-Prot", "homologene.id")
    
    gene_ensembls = genetable["ensembl.gene"]
    
    for gene_ensembl in gene_ensembls:
        variables0 = {"ensemblId": gene_ensembl}
        r0 = requests.post(base_url, json = {"query": query_geneinfo, "variables": variables0})
        res0 = json.loads(r0.text)
        if not res0["data"]["target"]:
            continue 
        else:
            break
    
    ensembl_gene_ok = pd.DataFrame(list(flatsplode(res0["data"]["target"])))["id"][0]
    
    genetable_ok = genetable >> \
        da.filter(da.f["ensembl.gene"] == ensembl_gene_ok) >> \
        da.rename(ensembl_gene_ID = "ensembl.gene", 
                  entrezgene_ID = "entrezgene", 
                  uniprot_ID = "uniprot.Swiss-Prot", 
                  homologene_ID = "homologene.id")
    
    return genetable_ok

In [308]:
genetable_ok_out = genetable_ok_def()

In [309]:
genetable_ok_out

,symbol,name,ensembl_gene_ID,entrezgene_ID,uniprot_ID,homologene_ID
,<object>,<object>,<object>,<object>,<object>,<int64>
1,IRF7,interferon regulatory factor 7,ENSG00000185507,3665,Q92985,128624


In [360]:
genetable_ok_out.shape

(1, 6)

In [310]:
type(genetable_ok_out)

datar.core.tibble.Tibble

## homologene

In [311]:
len(da.as_character(genetable_ok_out["homologene_ID"])) != 0

True

In [312]:
da.as_character(genetable_ok_out["homologene_ID"])

1    128624
Name: homologene_ID, dtype: object

In [313]:
type(da.as_character(genetable_ok_out["homologene_ID"]))

pandas.core.series.Series

### mouse

In [314]:
genetable_m0 = mg.query("homologene.id: " + da.as_character(genetable_ok_out["homologene_ID"]),
    fields = "symbol, name, ensembl.gene, entrezgene, uniprot.Swiss-Prot, homologene.id",
    species = "mouse")

In [315]:
genetable_m0["total"] == 0

False

In [316]:
genetable_m = pd.concat([genetable00, pd.DataFrame(list(flatsplode(genetable_m0["hits"][0])))]) >> \
    da.select("symbol", "name", "ensembl.gene", "entrezgene", "uniprot.Swiss-Prot", "homologene.id") >> \
    da.rename(ensembl_gene_ID = "ensembl.gene", 
              entrezgene_ID = "entrezgene", 
              uniprot_ID = "uniprot.Swiss-Prot", 
              homologene_ID = "homologene.id")

In [317]:
genetable_m 

,symbol,name,ensembl_gene_ID,entrezgene_ID,uniprot_ID,homologene_ID
,<object>,<object>,<object>,<object>,<object>,<int64>
0,Irf7,interferon regulatory factor 7,ENSMUSG00000025498,54123,P70434,128624


In [361]:
genetable_m.shape

(1, 6)

In [362]:
type(genetable_m)

pandas.core.frame.DataFrame

### rat

In [319]:
genetable_r0 = mg.query("homologene.id: " + da.as_character(genetable_ok_out["homologene_ID"]),
    fields = "symbol, name, ensembl.gene, entrezgene, uniprot.Swiss-Prot, homologene.id",
    species = "rat")

In [320]:
genetable_r0["total"] == 0

False

In [321]:
genetable_r = pd.concat([genetable00, pd.DataFrame(list(flatsplode(genetable_r0["hits"][0])))]) >> \
    da.select("symbol", "name", "ensembl.gene", "entrezgene", "uniprot.Swiss-Prot", "homologene.id") >> \
    da.rename(ensembl_gene_ID = "ensembl.gene", 
              entrezgene_ID = "entrezgene", 
              uniprot_ID = "uniprot.Swiss-Prot", 
              homologene_ID = "homologene.id")

In [322]:
genetable_r

,symbol,name,ensembl_gene_ID,entrezgene_ID,uniprot_ID,homologene_ID
,<object>,<object>,<object>,<object>,<object>,<int64>
0,Irf7,interferon regulatory factor 7,ENSRNOG00000017414,293624,NaN,128624


In [363]:
genetable_r.shape

(1, 6)

In [323]:
type(genetable_r)

pandas.core.frame.DataFrame

# score

## disease size

In [324]:
query_associatedDiseases_info = """
    query target($ensemblId: String!) {
        target(ensemblId: $ensemblId) {
            associatedDiseases(enableIndirect: false) {
                count
            }
        }
    }
"""

In [325]:
variables01 = {"ensemblId": ensembl_gene_ok}

In [326]:
r01 = requests.post(base_url, json = {"query": query_associatedDiseases_info, "variables": variables01})

In [327]:
res01 = json.loads(r01.text)

In [328]:
disease_size = res01["data"]["target"]["associatedDiseases"]["count"]

In [329]:
disease_size

274

In [330]:
type(disease_size)

int

## overall

In [331]:
query_associatedDiseases_scores = """
    query target($ensemblId: String!, $size: Int!) {
        target(ensemblId: $ensemblId) {
            associatedDiseases(enableIndirect: false, page: {index: 0, size: $size}) {
                rows {
                    score
                    disease {
                        id
                        name
                    }
                }
            }
        }
    }
"""

In [332]:
variables02 = {"ensemblId": ensembl_gene_ok, "size": disease_size}

In [333]:
r02 = requests.post(base_url, json = {"query": query_associatedDiseases_scores, "variables": variables02})

In [334]:
res02 = json.loads(r02.text)

In [335]:
overall_table = pd.DataFrame(list(flatsplode(res02["data"]["target"]["associatedDiseases"]))) >> \
    da.select("rows.disease.id", "rows.disease.name", "rows.score") >> \
    da.rename(disease_ID = "rows.disease.id", 
              disease_name = "rows.disease.name", 
              overall_score = "rows.score")

In [336]:
overall_table.head()

,disease_ID,disease_name,overall_score
,<object>,<object>,<float64>
0,MONDO_0015823,primary immunodeficiency due to a defect in ad...,0.464244
1,MONDO_0007915,systemic lupus erythematosus,0.337470
2,EFO_0004705,hypothyroidism,0.218722
3,EFO_0000571,lung adenocarcinoma,0.189006
4,MONDO_0011962,endometrial cancer,0.184790


In [337]:
overall_table.shape

(274, 3)

In [338]:
type(overall_table)

pandas.core.frame.DataFrame

## datatype

In [339]:
query_associatedDiseases_scores_dt = """
    query target($ensemblId: String!, $size: Int!) {
        target(ensemblId: $ensemblId) {
            associatedDiseases(enableIndirect: false, page: {index: 0, size: $size}) {
                rows {
                    datatypeScores {
                        id
                        score
                    }
                    disease {
                        id
                        name
                    }
                }
            }
        }
    }
"""

In [340]:
r03 = requests.post(base_url, json = {"query": query_associatedDiseases_scores_dt, "variables": variables02})

In [341]:
res03 = json.loads(r03.text)

In [342]:
dt_table = pd.DataFrame(list(flatsplode(res03["data"]["target"]["associatedDiseases"]))) >> \
    da.select("rows.disease.id", "rows.disease.name", "rows.datatypeScores.id", "rows.datatypeScores.score") >> \
    da.rename(disease_ID = "rows.disease.id", 
              disease_name = "rows.disease.name", 
              datatype = "rows.datatypeScores.id", 
              datatype_score = "rows.datatypeScores.score") >> \
    da.pivot_wider(names_from = da.f.datatype, values_from = da.f.datatype_score)

In [343]:
dt_table.head()

,disease_ID,disease_name,affected_pathway,animal_model,genetic_association,literature,rna_expression
,<object>,<object>,<float64>,<float64>,<float64>,<float64>,<float64>
0,DOID_7551,gonorrhea,NaN,NaN,NaN,0.042555,NaN
1,EFO_0000180,HIV-1 infection,NaN,NaN,NaN,0.205184,NaN
2,EFO_0000181,head and neck squamous cell carcinoma,0.330597,NaN,NaN,NaN,NaN
3,EFO_0000182,hepatocellular carcinoma,NaN,NaN,NaN,0.080995,NaN
4,EFO_0000220,acute lymphoblastic leukemia,NaN,NaN,NaN,0.012159,NaN


In [344]:
dt_table.shape

(274, 7)

In [345]:
type(dt_table)

datar.core.tibble.Tibble

In [380]:
dt_table00 = pd.DataFrame({
    "disease_ID": pd.Series([], dtype = object), 
    "disease_name": pd.Series([], dtype = object), 
    
    "known_drug": pd.Series([], dtype = np.float64), 
    "genetic_association": pd.Series([], dtype = np.float64), 
    "animal_model": pd.Series([], dtype = np.float64), 
    "literature": pd.Series([], dtype = np.float64), 
    "rna_expression": pd.Series([], dtype = np.float64), 
    "affected_pathway": pd.Series([], dtype = np.float64), 
    "somatic_mutation": pd.Series([], dtype = np.float64)
    }
)

In [348]:
dt_table00

,disease_ID,disease_name,known_drug,genetic_association,animal_model,literature,rna_expression,affected_pathway,somatic_mutation
,<object>,<object>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>


In [349]:
dt_table00.shape

(0, 9)

In [350]:
type(dt_table00)

pandas.core.frame.DataFrame

In [365]:
dt_table_ok = pd.concat([dt_table00, dt_table]) >> \
    da.replace_na(dict(known_drug = 0, genetic_association = 0, animal_model = 0, 
                       literature = 0, rna_expression = 0, 
                       affected_pathway = 0, somatic_mutation = 0))

In [367]:
dt_table_ok.head()

,disease_ID,disease_name,known_drug,genetic_association,animal_model,literature,rna_expression,affected_pathway,somatic_mutation
,<object>,<object>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>
0,DOID_7551,gonorrhea,0.0,0.0,0.0,0.042555,0.0,0.000000,0.0
1,EFO_0000180,HIV-1 infection,0.0,0.0,0.0,0.205184,0.0,0.000000,0.0
2,EFO_0000181,head and neck squamous cell carcinoma,0.0,0.0,0.0,0.000000,0.0,0.330597,0.0
3,EFO_0000182,hepatocellular carcinoma,0.0,0.0,0.0,0.080995,0.0,0.000000,0.0
4,EFO_0000220,acute lymphoblastic leukemia,0.0,0.0,0.0,0.012159,0.0,0.000000,0.0


In [368]:
dt_table_ok.shape

(274, 9)

In [369]:
type(dt_table_ok)

pandas.core.frame.DataFrame

## datasource

In [370]:
query_associatedDiseases_scores_ds = """
    query target($ensemblId: String!, $size: Int!) {
        target(ensemblId: $ensemblId) {
            associatedDiseases(enableIndirect: false, page: {index: 0, size: $size}) {
                rows {
                    datasourceScores {
                        id
                        score
                    }
                    disease {
                        id
                        name
                    }
                }
            }
        }
    }
"""

In [371]:
r04 = requests.post(base_url, json = {"query": query_associatedDiseases_scores_ds, "variables": variables02})

In [372]:
res04 = json.loads(r04.text)

In [376]:
ds_table = pd.DataFrame(list(flatsplode(res04["data"]["target"]["associatedDiseases"]))) >> \
    da.select("rows.disease.id", "rows.disease.name", "rows.datasourceScores.id", "rows.datasourceScores.score") >> \
    da.rename(disease_ID = "rows.disease.id", 
              disease_name = "rows.disease.name", 
              datasource = "rows.datasourceScores.id", 
              datasource_score = "rows.datasourceScores.score") >> \
    da.pivot_wider(names_from = da.f.datasource, values_from = da.f.datasource_score)

In [377]:
ds_table.head()

,disease_ID,disease_name,europepmc,eva,expression_atlas,impc,ot_genetics_portal,slapenrich
,<object>,<object>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>
0,DOID_7551,gonorrhea,0.042555,NaN,NaN,NaN,NaN,NaN
1,EFO_0000180,HIV-1 infection,0.205184,NaN,NaN,NaN,NaN,NaN
2,EFO_0000181,head and neck squamous cell carcinoma,NaN,NaN,NaN,NaN,NaN,0.330597
3,EFO_0000182,hepatocellular carcinoma,0.080995,NaN,NaN,NaN,NaN,NaN
4,EFO_0000220,acute lymphoblastic leukemia,0.012159,NaN,NaN,NaN,NaN,NaN


In [378]:
ds_table.shape

(274, 8)

In [379]:
type(ds_table)

datar.core.tibble.Tibble

In [385]:
ds_table00 = pd.DataFrame({
    "disease_ID": pd.Series([], dtype = object), 
    "disease_name": pd.Series([], dtype = object), 
    
    "chembl": pd.Series([], dtype = np.float64), 
    
    "ot_genetics_portal": pd.Series([], dtype = np.float64), 
    "phewas_catalog": pd.Series([], dtype = np.float64), 
    
    "eva": pd.Series([], dtype = np.float64), 
    "genomics_england": pd.Series([], dtype = np.float64), 
    "genephenotype": pd.Series([], dtype = np.float64), 
    "uniprot_literature": pd.Series([], dtype = np.float64), 
    "uniprot_variants": pd.Series([], dtype = np.float64), 
    "clingen": pd.Series([], dtype = np.float64), 
    "orphanet": pd.Series([], dtype = np.float64), 
    
    "impc": pd.Series([], dtype = np.float64), 
    
    "europepmc": pd.Series([], dtype = np.float64), 
    
    "expression_atlas": pd.Series([], dtype = np.float64), 
    
    "crispr": pd.Series([], dtype = np.float64), 
    "cancer_biomarkers": pd.Series([], dtype = np.float64), 
    "slapenrich": pd.Series([], dtype = np.float64), 
    "progeny": pd.Series([], dtype = np.float64), 
    
    "reactome": pd.Series([], dtype = np.float64), 
    "sysbio": pd.Series([], dtype = np.float64), 
    
    "cancer_gene_census": pd.Series([], dtype = np.float64), 
    "intogen": pd.Series([], dtype = np.float64), 
    "eva_somatic": pd.Series([], dtype = np.float64)
    }
)

In [386]:
ds_table00

,disease_ID,disease_name,chembl,ot_genetics_portal,phewas_catalog,eva,genomics_england,genephenotype,uniprot_literature,uniprot_variants,...,expression_atlas,crispr,cancer_biomarkers,slapenrich,progeny,reactome,sysbio,cancer_gene_census,intogen,eva_somatic
,<object>,<object>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,...,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>


In [383]:
ds_table00.shape

(0, 24)

In [384]:
type(ds_table00)

pandas.core.frame.DataFrame

In [387]:
ds_table_ok = pd.concat([ds_table00, ds_table]) >> \
    da.replace_na(dict(chembl = 0, 
                       ot_genetics_portal = 0, phewas_catalog = 0, 
                       eva = 0, genomics_england = 0, genephenotype = 0, uniprot_literature = 0, 
                       uniprot_variants = 0, clingen = 0, orphanet = 0, 
                       impc = 0, 
                       europepmc = 0, 
                       expression_atlas = 0, 
                       crispr = 0, cancer_biomarkers = 0, slapenrich = 0, progeny = 0, 
                       reactome = 0, sysbio = 0, 
                       cancer_gene_census = 0, intogen = 0, eva_somatic = 0))

In [388]:
ds_table_ok.head()

,disease_ID,disease_name,chembl,ot_genetics_portal,phewas_catalog,eva,genomics_england,genephenotype,uniprot_literature,uniprot_variants,...,expression_atlas,crispr,cancer_biomarkers,slapenrich,progeny,reactome,sysbio,cancer_gene_census,intogen,eva_somatic
,<object>,<object>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,...,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>
0,DOID_7551,gonorrhea,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,EFO_0000180,HIV-1 infection,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,EFO_0000181,head and neck squamous cell carcinoma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.330597,0.0,0.0,0.0,0.0,0.0,0.0
3,EFO_0000182,hepatocellular carcinoma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,EFO_0000220,acute lymphoblastic leukemia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [389]:
ds_table_ok.shape

(274, 24)

In [390]:
type(ds_table_ok)

pandas.core.frame.DataFrame